In [1]:
using LinearAlgebra

In [2]:
function potential_energy(X)
    # Compute the potential energy of the particle configuration
    n = size(X,2)
    energy = 0.0
    for i in 1:n-1
        for j in i+1:n
            r = norm(X[:,i] - X[:,j])
            energy += 1 / r^2
        end
    end
    return energy
end

potential_energy (generic function with 1 method)

In [3]:
function gradient(X)
    # Compute the gradient of the potential energy
    k = size(X,1)
    n = size(X,2)
    G = zeros(k, n)
    for i in 1:n-1
        for j in i+1:n
            r = norm(X[:,i] - X[:,j])
            G[:,i] += 2*(X[:,i] - X[:,j]) / r^4
            G[:,j] += 2*(X[:,j] - X[:,i]) / r^4
        end
    end
    return G
end

gradient (generic function with 1 method)

In [4]:
function hessian(X)
    # Compute the Hessian of the potential energy
    k = size(X,1)
    n = size(X,2)
    H = zeros(k*n, k*n)
    
    for i in 1:n-1
        for j in i+1:n
            r = norm(X[:,i] - X[:,j])
            R = X[:,i] - X[:,j]
            
            H[(i-1)*k+1:i*k, (i-1)*k+1:i*k] += 2*R*R' / r^4 - 8*Matrix{Float64}(I, k, k) / r^6
            H[(j-1)*k+1:j*k, (j-1)*k+1:j*k] += 2*R*R' / r^4 - 8*Matrix{Float64}(I, k, k) / r^6
            H[(i-1)*k+1:i*k, (j-1)*k+1:j*k] -= 2*R*R' / r^4 - 8*Matrix{Float64}(I, k, k) / r^6
            H[(j-1)*k+1:j*k, (i-1)*k+1:i*k] -= 2*R*R' / r^4 - 8*Matrix{Float64}(I, k, k) / r^6
    
        end
    end

    return H
end

hessian (generic function with 1 method)

In [5]:
function gradient_descent(X::Matrix, alpha::Float64, tol::Float64, max_iter::Int)
    # X = initial point
    # alpha = step size
    # tol = tolerance for convergence
    # max_iter = maximum number of iterations
    iter = 0
    
    while iter < max_iter
        G = gradient(X)
        X -= alpha .* G
        X = X ./ sqrt.(sum(X.^2, dims=1)) # re-normalize positions
    
        if norm(G) < tol
            print("Converged in ", iter, "iterations")
            break
        end
        
        iter += 1
    end
    
    return X
end

gradient_descent (generic function with 1 method)

In [6]:
function newton_method(X::Matrix, tol::Float64, max_iter::Int)
    k = size(X,1)
    n = size(X,2)
    iter = 0
    while iter < max_iter
        G = gradient(X)
        H = hessian(X)
        dX = - H \ G[:]
        X += reshape(dX, k, n)
        X = X ./ sqrt.(sum(X.^2, dims=1)) # re-normalize positions
        if norm(G) < tol
            print("Converged in ", iter, "iterations")
            break
        end
        iter += 1
    end

    return X
end

newton_method (generic function with 1 method)

In [90]:
function regularized_newton_method(X::Matrix, tol::Float64, max_iter::Int)
    k = size(X,1)
    n = size(X,2)
    iter = 0
    while iter < max_iter
        G = gradient(X)
        H = hessian(X)
        H += 1e-6 * Matrix{Float64}(I,k*n,k*n)
        dX = - H \ G[:]
        X += reshape(dX, k, n)
        X = X ./ sqrt.(sum(X.^2, dims=1))
        if norm(G) < tol
            print("Converged in ", iter, "iterations")
            break
        end
        iter += 1
    end
    return X
end

regularized_newton_method (generic function with 1 method)

In [8]:
function random_thomson_problem(k::Int, n::Int)
    # Initialize particle positions randomly on the unit sphere
    X = randn(k, n)
    X = X ./ sqrt.(sum(X.^2, dims=1))

    # Gradient Descent
    #X = gradient_descent(X, 0.001, 1e-6, 10000)
    
    # Newton's Method
    #X = newton_method(X, 1e-6, 1000)
    
    # Regularized Newton's Method
    X = regularized_newton_method(X, 1e-6, 10000)

    return X
end

random_thomson_problem (generic function with 1 method)

In [195]:
X = random_thomson_problem(3,2)
fx = potential_energy(X)
gx = gradient(X)
print("Energy: ", fx, "\nGrad: ", gx)
norm(gx)

LoadError: SingularException(6)

In [231]:
X = [0.0 0.0; 1 0.0; 0.0 1]
fx = potential_energy(X)
gx = norm(gradient(X))
print("Energy: ", fx, "\nGrad: ", gx)

Energy: 0.4999999999999999
Grad: 0.9999999999999998

In [219]:
X1 = regularized_newton_method(X, 1e-8, 1000)
fx1 = potential_energy(X1)
gx1 = norm(gradient(X1))
print("Point: ", X1, "\nEnergy: ", fx1, "\nGrad: ", gx1)

Point: [0.0 0.0; 0.7256306316883444 0.6880867734285924; 0.6880844325775538 0.7256284119524463]
Energy: 354.72418544496946
Grad: 18896.492247945243

In [252]:
norm(X[1:3])

1.0